In [21]:
import pandas as pd
import numpy as np

## Import data

In [22]:
# Results folder
folder = '../results/'
data_mm_df = pd.read_csv(folder + 'csv/results_CLM5_mm.csv')
data_pe_df = pd.read_csv(folder + 'csv/results_CLM5_pe.csv')

## Merge Data

In [23]:
# Combine multimarket and partial eq results
data_mm_df['Type'] = 'Multimarket'
data_pe_df['Type'] = 'Partial Equilibrium'
data_merged_df     = pd.concat([data_mm_df, data_pe_df])

In [24]:
# Fix crop labels
data_merged_df['Crop'] = data_merged_df['Crop'].apply(lambda x: x.capitalize())

In [25]:
## Convert quantities in units of calories (1 MT = 1,000,000,000,000)
data_merged_df['Calories_Original'] = 0

# Corn: https://fdc.nal.usda.gov/fdc-app.html#/food-details/168400/nutrients
data_merged_df.loc[data_merged_df['Crop'] == 'Corn', 
                   'Calories_Original'] = (data_merged_df.loc[data_merged_df['Crop'] == 'Corn', 'Quantity_Original']
                                           .apply(lambda x: x*(98/100)*(1e12)))

# Soybean: https://fdc.nal.usda.gov/fdc-app.html#/food-details/169282/nutrients
data_merged_df.loc[data_merged_df['Crop'] == 'Soybean', 
                   'Calories_Original'] = (data_merged_df.loc[data_merged_df['Crop'] == 'Soybean', 'Quantity_Original']
                                           .apply(lambda x: x*(147/100)*(1e12)))

data_merged_df['Calories_Produced'] = data_merged_df['Calories_Original'] * (1+data_merged_df['Percent_Quantity_Change'])



In [26]:
# Aggregate data
data_agg_df = data_merged_df.groupby(['Country', 'Type', 'Year']).sum().reset_index()

# Recompute changes in surplus
data_agg_df['Percent_Change_in_Consumer_Surplus'] = (data_agg_df['Change_in_Consumer_Surplus'] / 
                                                     data_agg_df['Consumer_Surplus_Original'])
data_agg_df['Percent_Change_in_Producer_Surplus'] = (data_agg_df['Change_in_Producer_Surplus'] / 
                                                     data_agg_df['Producer_Surplus_Original'])
data_agg_df['Percent_Change_in_Total_Surplus']    = ((data_agg_df['Change_in_Producer_Surplus'] 
                                                     + data_agg_df['Change_in_Consumer_Surplus']) / 
                                                    (data_agg_df['Producer_Surplus_Original']
                                                     + data_agg_df['Consumer_Surplus_Original']))

data_agg_df['Percent_Calorie_Change'] = (data_agg_df['Calories_Produced']/data_agg_df['Calories_Original']) - 1

In [27]:
data_agg_df.head()

,Country,Type,Year,Percent_Price_Change,Percent_Quantity_Change,Welfare_Transfer_to_Producer,Welfare_lost_by_Consumer,Welfare_lost_by_Producer,Change_in_Producer_Surplus,Change_in_Consumer_Surplus,Percent_Change_in_Producer_Surplus,Percent_Change_in_Consumer_Surplus,Producer_Surplus_Original,Consumer_Surplus_Original,Quantity_Original,Calories_Original,Calories_Produced,Percent_Change_in_Total_Surplus,Percent_Calorie_Change
0,Argentina,Multimarket,1,-0.092679,-0.166768,-1.017045e+09,-4.085920e+07,1.506018e+09,-2.523063e+09,1.338885e+10,0.018387,-0.097574,-1.372180e+11,-1.372180e+11,86484880,1.109201e+21,1.010387e+21,-0.039593,-0.089086
1,Argentina,Multimarket,2,-0.079839,-0.239251,-6.046442e+08,-3.810605e+07,2.672083e+09,-3.276728e+09,2.082968e+10,0.023880,-0.151800,-1.372180e+11,-1.372180e+11,86484880,1.109201e+21,9.584839e+20,-0.063960,-0.135879
2,Argentina,Multimarket,3,-0.056533,-0.164158,-3.473080e+08,-1.032582e+07,1.816911e+09,-2.164219e+09,1.400091e+10,0.015772,-0.102034,-1.372180e+11,-1.372180e+11,86484880,1.109201e+21,1.006917e+21,-0.043131,-0.092214
3,Argentina,Multimarket,4,-0.122740,-0.155815,-1.110350e+09,-3.773215e+07,1.414334e+09,-2.524684e+09,1.372145e+10,0.018399,-0.099997,-1.372180e+11,-1.372180e+11,86484880,1.109201e+21,1.011907e+21,-0.040799,-0.087715
4,Argentina,Multimarket,5,-0.051579,-0.136089,-3.009116e+08,-6.383966e+06,1.442505e+09,-1.743416e+09,1.116005e+10,0.012705,-0.081331,-1.372180e+11,-1.372180e+11,86484880,1.109201e+21,1.026391e+21,-0.034313,-0.074657


## Generate Tables

In [28]:
output_file = '../results/xlsx/results_CLM5_formatted.xlsx'
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

In [29]:
# Producer and Consumer Surplus Changes
country_subset = ['India', 'United States', 'China', 'Brazil']
for country in country_subset:
    
    ## Create table
    pivot_table_df = (data_agg_df.query('Country == @country')
                     .pivot_table(index = 'Year', columns = ['Type'], 
                                  values = ['Percent_Change_in_Consumer_Surplus', 'Percent_Change_in_Producer_Surplus', 
                                  'Percent_Change_in_Total_Surplus']))
    
    pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))
    
    # Abbreviate columns
    pivot_table_df.rename(columns = {'Multimarket':'MM', 'Partial Equilibrium': 'PE',
                                'Percent_Change_in_Consumer_Surplus': '% Change in Consumer Surplus',
                                'Percent_Change_in_Producer_Surplus': '% Change in Producer Surplus',
                                'Percent_Change_in_Total_Surplus': '% Change in Total Surplus'},
                         inplace = True)
    
    
    # Convert table to excel
    pivot_table_df.to_excel(writer, sheet_name = 'All_' + country, startrow = 1)
    
    ## Formatting
    worksheet = writer.sheets['All_' + country]

    # Add header line
    #header_line_label = col.split('_')[1]
    worksheet.merge_range(0, 0, 0, np.shape(pivot_table_df)[1], 
                                          country.capitalize() + ' -  % Change in Surplus for All Crops')

    # Remove blank row and country label
    worksheet.set_row(3, None, None, {'hidden': True})
    worksheet.write(1, 0, ' ')


    # Fix year label
    cell_format = writer.book.add_format()
    cell_format.set_bold()
    worksheet.write(2, 0, 'Year',  cell_format)

In [30]:
# Price and Quantity Changes
for crop in data_merged_df['Crop'].unique():
    for col in ['Percent_Price_Change', 'Percent_Quantity_Change']:
        
        ## Create table
        pivot_table_df = (data_merged_df.query('Crop == @crop & Country in @country_subset')
                          .pivot_table(index = 'Year', columns = ['Country', 'Type'], values = col)
                         )
        pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))
        
        # Abbreviate columns
        pivot_table_df.rename(columns = {'Multimarket':'MM', 'Partial Equilibrium': 'PE'},
                             inplace = True)
        
        # Convert table to excel
        pivot_table_df.to_excel(writer, sheet_name = crop + '_' + col, startrow = 1)
        
        ## Formatting
        worksheet = writer.sheets[crop + '_' + col]

        # Add header line
        header_line_label = col.split('_')[1]
        worksheet.merge_range(0, 0, 0, np.shape(pivot_table_df)[1], 
                                              crop + ' - % Change in ' + header_line_label + ' for each Country')

        # Remove blank row and country label
        worksheet.set_row(3, None, None, {'hidden': True})
        worksheet.write(1, 0, ' ')


        # Fix year label
        cell_format = writer.book.add_format()
        cell_format.set_bold()
        worksheet.write(2, 0, 'Year',  cell_format)

In [31]:
# Calorie Changes Country
col = 'Percent_Calorie_Change'
        
## Create table
pivot_table_df = (data_agg_df.query('Country in @country_subset')
                  .pivot_table(index = 'Year', columns = ['Country', 'Type'], values = col)
                 )
pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))

# Abbreviate columns
pivot_table_df.rename(columns = {'Multimarket':'MM', 'Partial Equilibrium': 'PE'},
                     inplace = True)

# Convert table to excel
pivot_table_df.to_excel(writer, sheet_name = col + '_Country', startrow = 1)

## Formatting
worksheet = writer.sheets[col + '_Country']

# Add header line
header_line_label = col.split('_')[1]
worksheet.merge_range(0, 0, 0, np.shape(pivot_table_df)[1], 
                                      '% Change in Calories for each Country')

# Remove blank row and country label
worksheet.set_row(3, None, None, {'hidden': True})
worksheet.write(1, 0, ' ')


# Fix year label
cell_format = writer.book.add_format()
cell_format.set_bold()
worksheet.write(2, 0, 'Year',  cell_format);

In [32]:
# Calorie Changes All
col = 'Percent_Calorie_Change'
        
# Create aggregate dataframe
data_agg_all_df = data_agg_df.groupby(['Type', 'Year']).sum().reset_index()
data_agg_all_df['Percent_Calorie_Change'] = (data_agg_all_df['Calories_Produced']/data_agg_all_df['Calories_Original']) - 1
    
## Create table
pivot_table_df = (data_agg_all_df.pivot_table(index = 'Year', columns = ['Type'], values = col))
pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))

# Abbreviate columns
pivot_table_df.rename(columns = {'Multimarket':'MM', 'Partial Equilibrium': 'PE'},
                     inplace = True)

# Convert table to excel
pivot_table_df.to_excel(writer, sheet_name = col + '_All', startrow = 1)

## Formatting
worksheet = writer.sheets[col + '_All']

# Add header line
header_line_label = col.split('_')[1]
worksheet.merge_range(0, 0, 0, np.shape(pivot_table_df)[1], 
                                      '% Change in Calories for all Countries')

# # Remove blank row and country label
# worksheet.set_row(3, None, None, {'hidden': True})
# worksheet.write(1, 0, ' ')


# # Fix year label
# cell_format = writer.book.add_format()
# cell_format.set_bold()
# worksheet.write(2, 0, 'Year',  cell_format);

In [33]:
writer.close()